In [ ]:
import boto3
import json
from typing import Dict, Any
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

region = os.environ.get('AWS_REGION')
MODEL_ID = os.environ.get('MODEL_ID')
role_arn = os.environ.get('ROLE_ARN')
knowledge_base_id = os.environ.get('KNOWLEDGE_BASE_ID')

In [ ]:
bedrock_agent = boto3.client('bedrock-agent', region_name = region)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name = region)

In [ ]:
flow_name="KBSearcher"
flow_description="Simple RAG application searching Bedrock Knowledge Base."

In [ ]:
# The input node validates that the content of the InvokeFlow request is a JSON object.
input_node = {
    "type": "Input",
    "name": "FlowInput",
    "outputs": [
        {
            "name": "document",
            "type": "String"
        }
    ]
}

In [ ]:
### Knowledge base node
kb_search_node = {
        "name": "kb_search_node",
        "type": "KnowledgeBase",
        "configuration" : {
            'knowledgeBase': {
                'knowledgeBaseId': 'VLYHWSH50F',
                'modelId': MODEL_ID
            }
        },
        "inputs": [
            {
                "name": "retrievalQuery",
                "type": "String",
                "expression": "$.data"
            }
        ],
        "outputs": [
            {
                "name": "outputText",
                "type": "String"
            }
        ]
        
}

In [ ]:
output_node = {
    "type": "Output",
    "name": "FlowOutput",
    "inputs": [
        {
            "name": "document",
            "type": "String",
            "expression": "$.data"
        }
    ]
}

In [ ]:
connections =  [
    {
      "name": "Input_to_KB",
      "source": input_node['name'],
      "target": kb_search_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": kb_search_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "blog_to_output",
      "source": kb_search_node['name'],
      "target": output_node['name'],
      "type": "Data",
      "configuration": {
        "data": {
          "sourceOutput": kb_search_node['outputs'][0]['name'],
          "targetInput": output_node['inputs'][0]['name']
        }
      }
    }
]

In [ ]:
response = bedrock_agent.create_flow(
    name=flow_name,
    description=flow_description,
    executionRoleArn=role_arn,
    definition={
        "nodes": [input_node, kb_search_node, output_node],
        "connections": connections
    }
)

flow_id = response.get("id")

In [ ]:
bedrock_agent.prepare_flow(flowIdentifier=flow_id)

In [ ]:
response = bedrock_agent.create_flow_version(flowIdentifier=flow_id)
                                
flow_version = response.get("version")

In [ ]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier=flow_id,
    name="latest",
    description="Alias pointing to the latest version of the flow.",
    routingConfiguration=[
        {
            "flowVersion": flow_version
        }
    ]
)

flow_alias_id = response.get("id")

In [ ]:
client_runtime = boto3.client('bedrock-agent-runtime')

response = client_runtime.invoke_flow(
    flowIdentifier=flow_id,
    flowAliasIdentifier=flow_alias_id,
    inputs=[
        {
            "content": {
                "document": "What is agentic memory"
            },
            "nodeName": "FlowInput",
            "nodeOutputName": "document"
        }
    ]
)

result = {}

for event in response.get("responseStream"):
    result.update(event)

if result['flowCompletionEvent']['completionReason'] == 'SUCCESS':
    print("Flow invocation was successful! The output of the flow is as follows:\n")
    print(result['flowOutputEvent']['content']['document'])

else:
    print("The flow invocation completed because of the following reason:", result['flowCompletionEvent']['completionReason'])